In [1]:
import ee
import eemont
import geemap
from datetime import datetime, timedelta

# ECMWF Copernicus Atmosphere Monitoring Service (CAMS) in Indonesia

In [2]:
west = 90.0
east = 150.0
north = 15.0
south = -15.0

center_lat = (north + south)/2
center_lon = (west + east)/2
Map = geemap.Map(center = (center_lat, center_lon), zoom = 7)

In [5]:
obs = datetime.utcnow() - timedelta(days = 7)
end = obs + timedelta(days = 1)

print(f"Date: {obs.date().isoformat()}")

Date : 2021-07-09


ECMWF CAMS showing a mean PM2.5 forecast data from the current date. All units in ug/m^3. Only value above 40ug/m^3 is shown. 

In [4]:
bounds = ee.Geometry.BBox(west, south, east, north)
cams = ee.ImageCollection('ECMWF/CAMS/NRT') \
    .select('particulate_matter_d_less_than_25_um_surface') \
    .filterDate(obs.isoformat(), end.isoformat()) \
    .filter(ee.Filter.lt('model_forecast_hour', 24))

data = cams.mean().clip(bounds) * 1e9
mask = data.gt(40)
data = data.updateMask(mask)

vis_params = {
    'min' : 0,
    'max' : 300,
    'palette' : ['white', 'red']
}

Map.addLayer(data, vis_params, 'PM2.5 (ug/m^2)')
Map

Map(center=[0.0, 120.0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=…